In [1]:
from html.entities import name2codepoint

import numpy as np
import torch
from torch.optim import Adam
from torch.nn import Linear, ReLU, Dropout, BatchNorm1d
import os
import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime

from game import UltimateTicTacToeEnv

In [2]:
import logging

# Create logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)  # Capture all levels (DEBUG and above)

# Clear existing handlers (important in Jupyter to avoid duplicate logs)
logger.handlers.clear()
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

logger.addHandler(console_handler)

logger.debug("Debug message: good for step-by-step inspection.")
logger.info("Info message: general updates.")
logger.warning("Warning message: something might be wrong.")


2025-08-15 09:24:03,463 - DEBUG - Debug message: good for step-by-step inspection.
2025-08-15 09:24:03,464 - INFO - Info message: general updates.
2025-08-15 09:24:03,465 - WARNING - Warning message: something might be wrong.


In [3]:


## agent
def processObs(observation):
    return np.array(observation[0] + observation[1] +[1 if i in observation[2] else 0 for i in range(9)]).flatten()

def convert_state_to_tensor(state):
    board, macroboard, valid_mask = state

    board = board.astype(np.float32)
    if macroboard.shape != (9,9):
        macroboard = np.repeat(macroboard, 3, axis=0).repeat(3, axis=1).astype(np.float32)
    valid_mask = valid_mask.astype(np.float32)
    # print(f"board: {board.shape}, macroboard: {macroboard.shape}, valid mask: {valid_mask.shape}")
    if macroboard.shape != (9,9):
        print(macroboard)
    
    stacked = np.stack([board, macroboard, valid_mask], axis=-1)  # Shape: (9, 9, 3)
    return stacked

def perspective_state(state, player):
    board, macro, valid_mask = state
    if player == -1:
        board = -board
        macro = -macro
    return board, macro, valid_mask

def encode_perspective(board, player):
    """
    Flip the board perspective so that 'player' is always 1.
    
    board: np.array shape (9,9) or (9,9,2)
    player: 1 or -1
    
    returns: board with current player's perspective
    """
    # If board is (9,9)
    if board.ndim == 2:
        return board * player  # flips sign if player == -1
    
    # If board is stacked (9,9,2)
    elif board.ndim == 3:
        # Assume first channel = current player positions, second = opponent
        if player == 1:
            return board.copy()
        else:
            return np.stack([board[:,:,1], board[:,:,0]], axis=-1)  # swap channels


class ReplayBuffer:
    def __init__(self, mem_size, input_shape):
        self.mem_size = mem_size
        self.mem_counter = 0

        self.state_memory = np.zeros((mem_size, *input_shape), dtype=np.float32)
        self.new_state_memory = np.zeros((mem_size, *input_shape), dtype=np.float32)
        self.action_memory = np.zeros(mem_size, dtype=np.int64)
        self.reward_memory = np.zeros(mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(mem_size, dtype=np.float32)

    def store_transition(self, state, action, reward, new_state, done):
        index = self.mem_counter % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = new_state
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = float(done)
        self.mem_counter += 1

    def sample_memory(self, batch_size):
        max_mem = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)

        return (self.state_memory[batch],
                self.action_memory[batch],
                self.reward_memory[batch],
                self.new_state_memory[batch],
                self.terminal_memory[batch])



class DQNetwork(nn.Module):
    def __init__(self, input_shape=(3, 9, 9), n_actions=81, learning_rate=0.00005):
        super(DQNetwork, self).__init__()
        c, h, w = input_shape[0], input_shape[1], input_shape[2]  # (channels, height, width)

        self.net = nn.Sequential(
            nn.Conv2d(c, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * h * w, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)  # One Q-value per cell (81 actions)
        )

        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.loss = nn.MSELoss()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

    def forward(self, x):
        # if len(x.shape) == 3:  # (H, W, C)
        #     x = x.permute(2, 0, 1).unsqueeze(0)  # → (1, C, H, W)
        # elif len(x.shape) == 4:  # (batch, H, W, C)
        #     x = x.permute(0, 3, 1, 2)  # → (batch, C, H, W)
        return self.net(x)
        
    # def save(self, name = "model_uttt"):
    #     torch.save(self,"./models/" + name + ".pt")

    def save(self, prefix="model"):
        # Get current time in DD_MM_YY_HH format
        timestamp = datetime.now().strftime("%d_%m_%y_%H")
        filename = f"./models/{prefix}_{timestamp}.pt"
        torch.save(self.state_dict(), filename)


class DQNAgent():
    def __init__(self, player=1, env=UltimateTicTacToeEnv(), loading=True, name=""):
        learning_rate = 0.0015
        gamma = 0.9
        batch_size = 64
        n_actions = env.action_space
        mem_size = 10_000
        min_memory_for_training = 1000
        epsilon = 1
        epsilon_dec = 0.995
        epsilon_min = 0.05

        self.gamma = gamma
        self.batch_size = batch_size
        self.n_actions = n_actions
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_dec = epsilon_dec
        self.mem_size = mem_size
        self.min_memory_for_training = min_memory_for_training
        self.it_counter = 0

        input_shape = (3, 9, 9)  # (C ,H, W,)
        self.q = DQNetwork(input_shape, n_actions, learning_rate) # online
        self.q_target = DQNetwork(input_shape, n_actions, learning_rate) # target
        self.q_target.load_state_dict(self.q.state_dict())

        self.target_update_freq = 1000
        self.replay_buffer = ReplayBuffer(mem_size, input_shape)

        if loading:
            self.q.load_state_dict(torch.load(name))
        else:
            # self.learnNN(env)
            pass

    def getAction(self, env, observation):
        observation = convert_state_to_tensor(observation)
        observation = torch.from_numpy(observation).float().permute(2, 0, 1).unsqueeze(0).to(self.q.device)
        q = self.q.forward(observation)
        action = int(torch.argmax(q))
    
        # Always filter invalid moves
        valid_actions = env.get_valid_actions()
        if action not in valid_actions:
            q_min = float(torch.min(q))
            mask = np.array([i in valid_actions for i in range(env.action_space)])
            new_q = (q.detach().cpu().numpy() - q_min + 1.) * mask
            action = int(np.argmax(new_q))
    
        return action
    
    def pickActionMaybeRandom(self, env, observation):
        if np.random.random() < self.epsilon:
            return int(np.random.choice(env.get_valid_actions()))
        else:
            return self.getAction(env, observation)
    
    def pickActionMaybeMasked(self, env, observation):
        # Now identical to pickActionMaybeRandom but with greedy fallback
        if np.random.random() < self.epsilon:
            return int(np.random.choice(env.get_valid_actions()))
        else:
            return self.getAction(env, observation)

    ## new - online and target
    def learn(self, error):
        if self.replay_buffer.mem_counter < self.min_memory_for_training:
            # print("NO LEARNING")
            # print(f"mem_counter = {self.replay_buffer.mem_counter}\n min_memory_for_training = {self.min_memory_for_training}")
            return
    
        states, actions, rewards, new_states, dones = self.replay_buffer.sample_memory(self.batch_size)
        
        # Convert to tensors
        states = torch.tensor(states, dtype=torch.float32).to(self.q.device)
        new_states = torch.tensor(new_states, dtype=torch.float32).to(self.q.device)
        actions = torch.tensor(actions, dtype=torch.long).to(self.q.device)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(self.q.device)
        dones = torch.tensor(dones, dtype=torch.float32).to(self.q.device)
    
        # Compute current Q-values (using online network)
        current_q = self.q(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Compute target Q-values (using target network)
        with torch.no_grad():  # No gradient for target computation
            next_q = self.q_target(new_states).max(1)[0]
            target_q = rewards + (1 - dones) * self.gamma * next_q
    
        # Compute loss and backpropagate
        loss = self.q.loss(current_q, target_q)
        self.q.optimizer.zero_grad()
        loss.backward()
        self.q.optimizer.step()
    
        # Periodically update target network
        if self.it_counter % self.target_update_freq == 0:
            self.q_target.load_state_dict(self.q.state_dict())
    
        # Adjust epsilon
        # if error == 0:
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_dec)
        # else:
            # self.epsilon = min(1.0, self.epsilon / self.epsilon_dec)

        # print(f"epsilon: {self.epsilon}")
        self.it_counter += 1

    
    # def learn(self, error):
    #     if self.replay_buffer.mem_counter < self.min_memory_for_training:
    #         return
            
    #     states, actions, rewards, new_states, dones = self.replay_buffer.sample_memory(self.batch_size)
    #     self.q.optimizer.zero_grad()
        
    #     # Convert to tensors and move to device
    #     states_batch = torch.tensor(states, dtype=torch.float32).to(self.q.device)
    #     new_states_batch = torch.tensor(new_states, dtype=torch.float32).to(self.q.device)
    #     actions_batch = torch.tensor(actions, dtype=torch.long).to(self.q.device)
    #     rewards_batch = torch.tensor(rewards, dtype=torch.float32).to(self.q.device)
    #     dones_batch = torch.tensor(dones, dtype=torch.float32).to(self.q.device)

    #     # Current Q values for chosen actions
    #     current_q = self.q.forward(states_batch).gather(1, actions_batch.unsqueeze(1)).squeeze(1)
        
    #     # Target Q values
    #     with torch.no_grad():
    #         next_q = self.q.forward(new_states_batch).max(1)[0]
    #         target_q = rewards_batch + (1 - dones_batch) * self.gamma * next_q

    #     # Compute loss and backpropagate
    #     loss = self.q.loss(current_q, target_q)
    #     loss.backward()
    #     self.q.optimizer.step()

    #     # Adjust epsilon
    #     if error == 0:
    #         self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_dec)
    #     else:
    #         self.epsilon = min(1.0, self.epsilon / self.epsilon_dec)

    #     self.it_counter += 1

    # def learn(self, error):
    #     if self.replay_buffer.mem_counter < self.min_memory_for_training:
    #         return
    #     states, actions, rewards, new_states, dones = self.replay_buffer.sample_memory(self.batch_size)
    #     self.q.optimizer.zero_grad()
    #     states_batch = torch.tensor(states, dtype = torch.float32).permute(2, 0, 1).unsqueeze(0).to(self.q.device)
    #     new_states_batch = torch.tensor(new_states,dtype = torch.float32).permute(2, 0, 1).unsqueeze(0).to(self.q.device)
        
    #     actions_batch = torch.tensor(actions, dtype = torch.long).to(self.q.device)
    #     rewards_batch = torch.tensor(rewards, dtype = torch.float32).to(self.q.device)
    #     dones_batch = torch.tensor(dones, dtype = torch.float32).to(self.q.device)

    #     target = rewards_batch + torch.mul(self.gamma* self.q(new_states_batch).max(axis = 1).values, (1 - dones_batch))
    #     prediction = self.q.forward(states_batch).gather(1,actions_batch.unsqueeze(1)).squeeze(1)

    #     loss = self.q.loss(prediction, target)
    #     loss.backward()  # Compute gradients
    #     self.q.optimizer.step()  # Backpropagate error

    #     # decrease epsilon:
    #     if error == 0:
    #         if self.epsilon * self.epsilon_dec > self.epsilon_min:
    #             self.epsilon *= self.epsilon_dec
    #     else:
    #         if self.epsilon / self.epsilon_dec <= 1:
    #             self.epsilon /= self.epsilon_dec

    #     self.it_counter += 1
    #     return

    
    def learnNN(self, env, n_episodes=10, n_save=1000, trainingName=""):
        for ep in tqdm.tqdm(range(n_episodes)):
            state = env.reset()
            done = False
            current_player = 1
            moves_sequence = []
            last_state = {1: None, -1: None}
            last_action = {1: None, -1: None}
            while not done:
                # Create player perspective state
                p_state = perspective_state(state, current_player)
                p_state_t = convert_state_to_tensor(p_state)

                # print(f"p_state:\n{p_state[0]}")
    
                valid_actions = env.get_valid_actions()
                if len(valid_actions) == 0:
                    logger.debug("WARNING: No valid actions found!")
                    logger.debug(f"moves: {moves_sequence}, len: {len(moves_sequence)}")
                    
                    break
    
                action = self.pickActionMaybeMasked(env, p_state) # zmiana na p_state
                moves_sequence.append(action)
                last_state[current_player] = p_state
                last_action[current_player] = action
                new_state, reward, done, error = env.step(action)
                # logger.debug(action)
                # Flip reward for perspective
                reward_p = reward if current_player == 1 else -reward

                # if reward_p != 0:
                # print(reward_p)
                
                
                # Store experience
                state_s = p_state_t.transpose(2, 0, 1)  # Convert to (C, H, W)
                state_ns = convert_state_to_tensor(perspective_state(new_state, current_player)).transpose(2, 0, 1)

                # print(f"state :\n{state_s[0]}")
                # print(f"new state :\n{state_ns[0]}")
                self.replay_buffer.store_transition(
                    state_s, action, reward_p, state_ns, done
                )
    
                # Learn
                self.learn(error)
    
                # Advance turn
                state = new_state
                current_player *= -1
                # print(f"-------{action}")
                # env.game.board.print_board()
                
            # logger.debug("restart")
            # Save checkpoint
            if (ep + 1) % n_save == 0:
                self.q.save(f"{trainingName}_{ep+1}")
        
        agent.q.save(prefix="ultimate_ttt_1m")
    

    # def learnNN(self, env, masked = True, n_episodes = 10000, n_save = 1000, trainingName = ""):
    #     l_epsilon = []
    #     l_win = []
    #     sum_win = 0
    #     i = 1
    #     for episode in tqdm.tqdm(range(n_episodes)):
    #         state = env.reset()         # resetting the environment after each episode
    #         score = 0
    #         done = 0
    #         while not done:               # while the episode is not over yet
    #             action = None
    #             if masked:
    #                 action = self.getAction(env, state)
    #             else:
    #                 action = self.pickActionMaybeMasked(env,state)           # let the agent act
    #             logging.debug(action)
    #             new_state,reward, done, error = env.step(action) # performing the action in the environment
    #             score += reward                            #  the total score during this round
    #             state = np.transpose(convert_state_to_tensor(state), (2, 0, 1))
    #             new_state = np.transpose(convert_state_to_tensor(new_state), (2, 0, 1))
    #             self.replay_buffer.store_transition(state, action, reward,new_state, done)   # store timestep for experiene replay
    #             # self.learn(error)                            # the agent learns after each timestep
    #             state = new_state
    #             # print(f"state learnNN: {state}")
    #             # print(i)
    #             i+=1
    #         if env.game.state == 1:
    #             sum_win +=1
    #         elif env.game.state == -1:
    #             sum_win -= 1
    #         elif env.game.state == 2:
    #             sum_win += 0

    #         l_epsilon.append(self.epsilon)
    #         l_win.append(sum_win)

    #         if (episode+1) % n_save == 0:
    #             self.q.save(trainingName + "_" + str(episode+1))

    #     # env.close()
    #     self.q.save(trainingName + "_final")
    #     print(l_epsilon)
    #     print("\n")
    #     print(l_win)

In [4]:
env = UltimateTicTacToeEnv()
env.action_space

81

In [5]:
state, reward, done, info = env.step(1)

In [6]:
observation = convert_state_to_tensor(state)
observation = torch.from_numpy(observation).float().permute(2, 0, 1).unsqueeze(0)
observation.shape

torch.Size([1, 3, 9, 9])

In [25]:
agent = DQNAgent(player=1, env=UltimateTicTacToeEnv(), loading=True, name = r"models/uttt_zero_1000_15_08_25_01.pt")
agent.learnNN(env, n_episodes=70_000, trainingName="uttt_zero")


  2%|█▍                                                                      | 1433/70000 [1:00:59<48:38:04,  2.55s/it]


KeyboardInterrupt: 

In [ ]:
moves = [68, 35, 24, 74, 69, 27, 19, 58, 23, 61, 14, 51, 63, 47, 71, 34, 13, 48, 56, 17, 43, 40, 30, 1, 21, 73, 75, 64, 50, 80, 79, 76, 66, 28, 4, 5, 15, 36, 46, 67, 32, 6, 11, 44, 52, 26, 60, 18, 55, 22, 54, 9, 37, 41, 53, 70, 39, 29, 8, 16, 31, 3, 20, 78, 2, 42, 62, 33, 12]

env = UltimateTicTacToeEnv()
for move in moves:
    state, reward, done, info = env.step(move)
    print(env.get_valid_actions())
    env.game.board.print_board()

In [ ]:
done

In [24]:
def play_against_ai(model_path=r"models/uttt_zero_5000_15_08_25_00.pt"):
    # Initialize environment and agent
    env = UltimateTicTacToeEnv()
    
    # Load model with weights_only=False (only do this for trusted models)
    agent = DQNAgent(player=1, env=env, loading=False, name=model_path)
# Load model weights properly
    state_dict = torch.load(model_path, weights_only=False)
    agent.q.load_state_dict(state_dict)  # load into the policy network
    agent.q_target.load_state_dict(agent.q.state_dict())  # sync target network

    
    # Rest of your code remains the same...
    
    # Set agent to evaluation mode (no exploration)
    agent.epsilon = 0  # Always choose best action
    
    # Game loop
    while True:
        # Human's turn
        env.render()
        print("\nYour turn (X)")
        valid_actions = env.get_valid_actions()
        print(f"Valid moves: {valid_actions}")
        
        while True:
            try:
                action = int(input("Enter your move (0-80): "))
                if action in valid_actions:
                    break
                print("Invalid move! Try again.")
            except ValueError:
                print("Please enter a number between 0-80")
        
        # Human makes move
        _, _, done, _ = env.step(action)
        if done:
            env.render()
            print("Game over!")
            if env.game.state == 1:
                print("You won!")
            elif env.game.state == -1:
                print("AI won!")
            else:
                print("It's a tie!")
            break
            
        # AI's turn
        env.render()
        print("\nAI's turn (O)")
        state = env._get_obs()
        action = agent.getAction(env, state)  # Get AI's move
        print(f"AI chooses: {action}")
        
        # AI makes move
        _, _, done, _ = env.step(action)
        if done:
            env.render()
            print("Game over!")
            if env.game.state == 1:
                print("You won!")
            elif env.game.state == -1:
                print("AI won!")
            else:
                print("It's a tie!")
            break

if __name__ == "__main__":
    # Example: play against the AI
    play_against_ai()  # Replace with your actual model path

· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·

Your turn (X)
Valid moves: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80]


Enter your move (0-80):  2


· · x · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·

AI's turn (O)
AI chooses: 25
· · x · · · · · ·
· · · · · · · · ·
· · · · · · · o ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·

Your turn (X)
Valid moves: [57 58 59 66 67 68 75 76 77]


Enter your move (0-80):  59


· · x · · · · · ·
· · · · · · · · ·
· · · · · · · o ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · x · · ·
· · · · · · · · ·
· · · · · · · · ·

AI's turn (O)
AI chooses: 15
· · x · · · · · ·
· · · · · · o · ·
· · · · · · · o ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · · · · ·
· · · · · x · · ·
· · · · · · · · ·
· · · · · · · · ·

Your turn (X)
Valid moves: [27 28 29 36 37 38 45 46 47]


...

x x x x · o x x x
x x x · o · x x x
x x x x · · x x x
x x x x o o o o ·
x x x x · o o · x
x x x · x · · x ·
o o o x x x x x x
o o o x x x x x x
o o o x x x x x x

AI's turn (O)
AI chooses: 53
x x x x · o x x x
x x x · o · x x x
x x x x · · x x x
x x x x o o o o ·
x x x x · o o · x
x x x · x · · x o
o o o x x x x x x
o o o x x x x x x
o o o x x x x x x

Your turn (X)
Valid moves: [ 4 12 14 22 23 35 40 43 48 50 51]


Enter your move (0-80):  51


x x x x · o x x x
x x x · o · x x x
x x x x · · x x x
x x x x o o o o ·
x x x x · o o · x
x x x · x · x x o
o o o x x x x x x
o o o x x x x x x
o o o x x x x x x

AI's turn (O)
AI chooses: 35
x x x x · o x x x
x x x · o · x x x
x x x x · · x x x
x x x x o o o o o
x x x x · o o o o
x x x · x · o o o
o o o x x x x x x
o o o x x x x x x
o o o x x x x x x

Your turn (X)
Valid moves: [ 4 12 14 22 23 40 48 50]


Enter your move (0-80):  12


x x x x x x x x x
x x x x x x x x x
x x x x x x x x x
x x x x o o o o o
x x x x · o o o o
x x x · x · o o o
o o o x x x x x x
o o o x x x x x x
o o o x x x x x x
Game over!
You won!


In [6]:
board

NameError: name 'board' is not defined